# ESCENARIO PRESENTE (2010)

# PAQUETES

In [1]:
import rasterio
import geopandas as gpd
import numpy as np
import pandas as pd
from rasterio.mask import mask
from shapely.geometry import box
import matplotlib.pyplot as plt

# CARGAR DATOS

In [2]:
#Cargar el grid
Grid_50 = gpd.read_file(r'Data/Grid50/Grid_50Km_Europe_WGS84.shp')
Grid_50

,ET_ID,COUNTRY,geometry
0,249,Spain,"POLYGON ((-2.18083 34.85145, -2.71695 34.77839..."
1,280,Malta,"POLYGON ((14.68659 35.53849, 14.14122 35.56449..."
2,363,Spain,"POLYGON ((-2.79499 35.22702, -3.33331 35.15029..."
3,412,Greece,"POLYGON ((23.95653 35.05738, 23.41935 35.13773..."
4,413,Greece,"POLYGON ((24.49271 34.97391, 23.95653 35.05738..."
...,...,...,...
3101,9369,Norway,"POLYGON ((26.92771 70.58708, 25.63446 70.69412..."
3102,9370,Norway,"POLYGON ((28.20775 70.47181, 26.92771 70.58708..."
3103,9371,Norway,"POLYGON ((29.47384 70.34844, 28.20775 70.47181..."
3104,9372,Norway,"POLYGON ((30.72534 70.21713, 29.47384 70.34844..."


In [3]:
# Abrir el raster y leer los datos fuera del bucle
src = rasterio.open(r'Data/GLOBIO/RCP2_6/Globio4_landuse_10sec_2050_SSP1RCP26_cropint_pasint.tif')

# Obtener la transformación y el sistema de referencia del raster
raster_transform = src.transform
raster_crs = src.crs


In [4]:
# Reproyectar la malla al sistema de coordenadas del raster si es necesario
if Grid_50.crs != raster_crs:
    Grid_50 = Grid_50.to_crs(raster_crs)

# PROCESAMIENTO DATOS DE HABITAT

Con los datos descargados de GLOBIO para los usos del suelo del año 2010 (periodo coincidente con los datos de temperatura) se procede a extraer la información asociada a cada categoría de hábitat definida por GLOBIO que sea de interés para el modelo de distribución de especies. Para el SDM se ha establecido 4 tipologías de habitas, las cuales están constituidas por distintas categorías de GLOBIO.

- Habitat 1 - Zonas acuáticas
- Habitat 2 - Vegetación Arbórea Perenne
- Habitat 3 - Vegetación Arbórea Caduca
- Habitat 4 - Matorrales y Pastizales


## Habitat 1 - Zonas acuáticas

El habitat 1, corresponderá con ambientes acuaticos, por lo que se incluyen las siguiente tipologias disponible en GLOBIO:
 - 160: Cubierta arbórea inundada con agua dulce o salobre
 - 170: Cubierta arbórea inundada con agua salina
 - 180: Cobertura de arbustos o herbácea inundada con agua dulce/salina/salobre
 - 210: Cuerpos de agua

In [5]:
def extraccion_calculo_area(code):
    # Crear una columna para almacenar el área del código específico por celda
    column_name = f'area_{code}'
    Grid_50[column_name] = 0.0
    
    # Iterar sobre cada celda en la malla
    for idx, cell in Grid_50.iterrows():
        # Crear una máscara para la celda actual
        geom = [cell.geometry]
        out_image, out_transform = mask(src, geom, crop=True, filled=False)
        
        # Extraer el código específico
        code_mask = (out_image == code).astype(np.uint8)
        
        # Calcular el área del código específico en la celda actual (en m²)
        area_code = np.sum(code_mask) * (raster_transform[0] ** 2)  # Área en m²
        
        # Almacenar el resultado en la celda correspondiente
        Grid_50.at[idx, column_name] = area_code

In [6]:
extraccion_calculo_area(160)
extraccion_calculo_area(170)
extraccion_calculo_area(180)
extraccion_calculo_area(210)

In [7]:
Grid_50

,ET_ID,COUNTRY,geometry,area_160,area_170,area_180,area_210
0,249,Spain,"POLYGON ((-2.18083 34.85145, -2.71695 34.77839...",0.000000,0.0,0.000000,0.087106
1,280,Malta,"POLYGON ((14.68659 35.53849, 14.14122 35.56449...",0.000000,0.0,0.000000,0.224738
2,363,Spain,"POLYGON ((-2.79499 35.22702, -3.33331 35.15029...",0.000000,0.0,0.000000,0.207083
3,412,Greece,"POLYGON ((23.95653 35.05738, 23.41935 35.13773...",0.000000,0.0,0.000039,0.113789
4,413,Greece,"POLYGON ((24.49271 34.97391, 23.95653 35.05738...",0.000000,0.0,0.000000,0.123380
...,...,...,...,...,...,...,...
3101,9369,Norway,"POLYGON ((26.92771 70.58708, 25.63446 70.69412...",0.000000,0.0,0.004144,0.441667
3102,9370,Norway,"POLYGON ((28.20775 70.47181, 26.92771 70.58708...",0.000000,0.0,0.010941,0.180910
3103,9371,Norway,"POLYGON ((29.47384 70.34844, 28.20775 70.47181...",0.000008,0.0,0.006057,0.130077
3104,9372,Norway,"POLYGON ((30.72534 70.21713, 29.47384 70.34844...",0.000000,0.0,0.003503,0.214846


In [8]:
# Seleccionar las columnas de especies (las que comienzan con 'sp_')
area_columns = [col for col in Grid_50.columns if col.startswith('area_')]
#Realizar el sumatorio de las distintas columnas con el area de cada tipología
Grid_50['Area_H_1'] = Grid_50[area_columns].sum(axis=1)

In [9]:
Grid_50.head()

,ET_ID,COUNTRY,geometry,area_160,area_170,area_180,area_210,Area_H_1
0,249,Spain,"POLYGON ((-2.18083 34.85145, -2.71695 34.77839...",0.0,0.0,0.000000,0.087106,0.087106
1,280,Malta,"POLYGON ((14.68659 35.53849, 14.14122 35.56449...",0.0,0.0,0.000000,0.224738,0.224738
2,363,Spain,"POLYGON ((-2.79499 35.22702, -3.33331 35.15029...",0.0,0.0,0.000000,0.207083,0.207083
3,412,Greece,"POLYGON ((23.95653 35.05738, 23.41935 35.13773...",0.0,0.0,0.000039,0.113789,0.113827
4,413,Greece,"POLYGON ((24.49271 34.97391, 23.95653 35.05738...",0.0,0.0,0.000000,0.123380,0.123380


In [10]:
# Guardar el resultado en un nuevo shapefile
#Grid_50.to_file('Data/GLOBIO/RCP2_6/Grid_H_1.shp')

In [11]:
Grid_50.columns

Index(['ET_ID', 'COUNTRY', 'geometry', 'area_160', 'area_170', 'area_180',
       'area_210', 'Area_H_1'],
      dtype='object')

In [12]:
columns_select = ['ET_ID', 'COUNTRY', 'geometry',
                  'Area_H_1'
                  ]

Grid_Habitats = Grid_50[columns_select]
Grid_Habitats

,ET_ID,COUNTRY,geometry,Area_H_1
0,249,Spain,"POLYGON ((-2.18083 34.85145, -2.71695 34.77839...",0.087106
1,280,Malta,"POLYGON ((14.68659 35.53849, 14.14122 35.56449...",0.224738
2,363,Spain,"POLYGON ((-2.79499 35.22702, -3.33331 35.15029...",0.207083
3,412,Greece,"POLYGON ((23.95653 35.05738, 23.41935 35.13773...",0.113827
4,413,Greece,"POLYGON ((24.49271 34.97391, 23.95653 35.05738...",0.123380
...,...,...,...,...
3101,9369,Norway,"POLYGON ((26.92771 70.58708, 25.63446 70.69412...",0.445810
3102,9370,Norway,"POLYGON ((28.20775 70.47181, 26.92771 70.58708...",0.191852
3103,9371,Norway,"POLYGON ((29.47384 70.34844, 28.20775 70.47181...",0.136142
3104,9372,Norway,"POLYGON ((30.72534 70.21713, 29.47384 70.34844...",0.218349


## Habitat 2 - Vegetación Arbórea Perenne

El habitat 2, corresponde con vegetación arbórea perenne, por lo que se incluyen las siguiente tipologias disponible en GLOBIO:

 - 50: Cubierta arbórea de hoja ancha perenne cerrada a abierta (>15%)
 - 70: Cubierta arbórea de hoja acicular perenne cerrada a abierta (>15%)
 - 71: Cubierta arbórea de hoja acicular perenne cerrada (>40%)
 - 72: Cubierta arbórea de hoja acicular perenne abierta (15-40%)

In [13]:
#Cargar el grid nuevamente
Grid_50 = gpd.read_file(r'Data/Grid50/Grid_50Km_Europe_WGS84.shp')
# Reproyectar la malla al sistema de coordenadas del raster si es necesario
if Grid_50.crs != raster_crs:
    Grid_50 = Grid_50.to_crs(raster_crs)

In [14]:
extraccion_calculo_area(50)
extraccion_calculo_area(70)
extraccion_calculo_area(71)
extraccion_calculo_area(72)

In [15]:
# Seleccionar las columnas de especies (las que comienzan con 'sp_')
area_columns = [col for col in Grid_50.columns if col.startswith('area_')]
#Realizar el sumatorio de las distintas columnas con el area de cada tipología
Grid_50['Area_H_2'] = Grid_50[area_columns].sum(axis=1)

In [16]:
Grid_50.head()

,ET_ID,COUNTRY,geometry,area_50,area_70,area_71,area_72,Area_H_2
0,249,Spain,"POLYGON ((-2.18083 34.85145, -2.71695 34.77839...",0.0,0.000046,0.0,0.0,0.000046
1,280,Malta,"POLYGON ((14.68659 35.53849, 14.14122 35.56449...",0.0,0.000000,0.0,0.0,0.000000
2,363,Spain,"POLYGON ((-2.79499 35.22702, -3.33331 35.15029...",0.0,0.000116,0.0,0.0,0.000116
3,412,Greece,"POLYGON ((23.95653 35.05738, 23.41935 35.13773...",0.0,0.004414,0.0,0.0,0.004414
4,413,Greece,"POLYGON ((24.49271 34.97391, 23.95653 35.05738...",0.0,0.001273,0.0,0.0,0.001273


In [17]:
# Guardar el resultado en un nuevo shapefile
#Grid_50.to_file('Data/GLOBIO/RCP2_6/Grid_H_2.shp')

In [18]:
Grid_50_s = Grid_50[['ET_ID', 'Area_H_2']]

In [19]:
#  Unir los DataFrames usando el campo ET_ID como clave
Grid_Habitats = pd.merge(Grid_Habitats, Grid_50_s, on='ET_ID', how='left')
Grid_Habitats

,ET_ID,COUNTRY,geometry,Area_H_1,Area_H_2
0,249,Spain,"POLYGON ((-2.18083 34.85145, -2.71695 34.77839...",0.087106,0.000046
1,280,Malta,"POLYGON ((14.68659 35.53849, 14.14122 35.56449...",0.224738,0.000000
2,363,Spain,"POLYGON ((-2.79499 35.22702, -3.33331 35.15029...",0.207083,0.000116
3,412,Greece,"POLYGON ((23.95653 35.05738, 23.41935 35.13773...",0.113827,0.004414
4,413,Greece,"POLYGON ((24.49271 34.97391, 23.95653 35.05738...",0.123380,0.001273
...,...,...,...,...,...
3101,9369,Norway,"POLYGON ((26.92771 70.58708, 25.63446 70.69412...",0.445810,0.003835
3102,9370,Norway,"POLYGON ((28.20775 70.47181, 26.92771 70.58708...",0.191852,0.017670
3103,9371,Norway,"POLYGON ((29.47384 70.34844, 28.20775 70.47181...",0.136142,0.007948
3104,9372,Norway,"POLYGON ((30.72534 70.21713, 29.47384 70.34844...",0.218349,0.003410


## Habitat 3 - Vegetación Arbórea Caduca

El habitat 3, corresponde con vegetación arbórea caduca, por lo que se incluyen las siguiente tipologias disponible en GLOBIO:

 - 60: Cubierta arbórea de hoja ancha caduca cerrada a abierta (>15%)
 - 61: Cubierta arbórea de hoja ancha caduca cerrada (>40%)
 - 62: Cubierta arbórea de hoja ancha caduca cerrada (>40%)
 - 80: Cubierta arbórea de hoja acicular caduca cerrada a abierta (>15%)
 - 81: Cubierta arbórea de hoja acicular caduca cerrada (>40%)
 - 82: Cubierta arbórea de hoja acicular caduca abierta (15-40%)

In [20]:
#Cargar el grid nuevamente
Grid_50 = gpd.read_file(r'Data/Grid50/Grid_50Km_Europe_WGS84.shp')
# Reproyectar la malla al sistema de coordenadas del raster si es necesario
if Grid_50.crs != raster_crs:
    Grid_50 = Grid_50.to_crs(raster_crs)

extraccion_calculo_area(60)
extraccion_calculo_area(61)
extraccion_calculo_area(62)
extraccion_calculo_area(80)
extraccion_calculo_area(81)
extraccion_calculo_area(82)

# Seleccionar las columnas de especies (las que comienzan con 'sp_')
area_columns = [col for col in Grid_50.columns if col.startswith('area_')]
#Realizar el sumatorio de las distintas columnas con el area de cada tipología
Grid_50['Area_H_3'] = Grid_50[area_columns].sum(axis=1)

Grid_50.head()

,ET_ID,COUNTRY,geometry,area_60,area_61,area_62,area_80,area_81,area_82,Area_H_3
0,249,Spain,"POLYGON ((-2.18083 34.85145, -2.71695 34.77839...",0.000177,0.0,0.0,0.0,0.0,0.0,0.000177
1,280,Malta,"POLYGON ((14.68659 35.53849, 14.14122 35.56449...",0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
2,363,Spain,"POLYGON ((-2.79499 35.22702, -3.33331 35.15029...",0.000031,0.0,0.0,0.0,0.0,0.0,0.000031
3,412,Greece,"POLYGON ((23.95653 35.05738, 23.41935 35.13773...",0.000139,0.0,0.0,0.0,0.0,0.0,0.000139
4,413,Greece,"POLYGON ((24.49271 34.97391, 23.95653 35.05738...",0.000000,0.0,0.0,0.0,0.0,0.0,0.000000


In [21]:
# Guardar el resultado en un nuevo shapefile
#Grid_50.to_file('Data/GLOBIO/RCP2_6/Grid_H_3.shp')

In [22]:
# Guardar el resultado en un nuevo shapefile
Grid_50_s = Grid_50[['ET_ID', 'Area_H_3']]

#  Unir los DataFrames usando el campo ET_ID como clave
Grid_Habitats = pd.merge(Grid_Habitats, Grid_50_s, on='ET_ID', how='left')
Grid_Habitats

,ET_ID,COUNTRY,geometry,Area_H_1,Area_H_2,Area_H_3
0,249,Spain,"POLYGON ((-2.18083 34.85145, -2.71695 34.77839...",0.087106,0.000046,0.000177
1,280,Malta,"POLYGON ((14.68659 35.53849, 14.14122 35.56449...",0.224738,0.000000,0.000000
2,363,Spain,"POLYGON ((-2.79499 35.22702, -3.33331 35.15029...",0.207083,0.000116,0.000031
3,412,Greece,"POLYGON ((23.95653 35.05738, 23.41935 35.13773...",0.113827,0.004414,0.000139
4,413,Greece,"POLYGON ((24.49271 34.97391, 23.95653 35.05738...",0.123380,0.001273,0.000000
...,...,...,...,...,...,...
3101,9369,Norway,"POLYGON ((26.92771 70.58708, 25.63446 70.69412...",0.445810,0.003835,0.004298
3102,9370,Norway,"POLYGON ((28.20775 70.47181, 26.92771 70.58708...",0.191852,0.017670,0.016150
3103,9371,Norway,"POLYGON ((29.47384 70.34844, 28.20775 70.47181...",0.136142,0.007948,0.010324
3104,9372,Norway,"POLYGON ((30.72534 70.21713, 29.47384 70.34844...",0.218349,0.003410,0.003356


## Habitat 4 - Matorrales y Pastizales

El habitat 4, corresponde con matorrales y pastizales, por lo que se incluyen las siguiente tipologias disponible en GLOBIO:

 - 2: Tierras de cultivo
 - 3: Pastizales en general
 - 4: Pastizales utilizados para la cría de ganado y la agricultura extensiva en zonas de pastoreo
 - 120: Matorrales
 - 121: Matorrales perennes
 - 122: Matorrales caducifolios
 - 130: Pastizales
 - 150: Vegetación dispersa (árboles, arbustos, cobertura herbácea) (<15%)
 - 152: Arbustos dispersos (<15%)
 - 153: Cobertura herbácea dispersa (<15%)
 - 110: Mosaico de cobertura herbácea (>50%) / árboles y arbustos (<50%)

In [23]:
#Cargar el grid nuevamente
Grid_50 = gpd.read_file(r'Data/Grid50/Grid_50Km_Europe_WGS84.shp')
# Reproyectar la malla al sistema de coordenadas del raster si es necesario
if Grid_50.crs != raster_crs:
    Grid_50 = Grid_50.to_crs(raster_crs)

extraccion_calculo_area(2)
extraccion_calculo_area(3)
extraccion_calculo_area(4)
extraccion_calculo_area(120)
extraccion_calculo_area(121)
extraccion_calculo_area(122)
extraccion_calculo_area(130)
extraccion_calculo_area(150)
extraccion_calculo_area(152)
extraccion_calculo_area(153)
extraccion_calculo_area(110)

# Seleccionar las columnas de especies (las que comienzan con 'sp_')
area_columns = [col for col in Grid_50.columns if col.startswith('area_')]
#Realizar el sumatorio de las distintas columnas con el area de cada tipología
Grid_50['Area_H_4'] = Grid_50[area_columns].sum(axis=1)

Grid_50.head()

,ET_ID,COUNTRY,geometry,area_2,area_3,area_4,area_120,area_121,area_122,area_130,area_150,area_152,area_153,area_110,Area_H_4
0,249,Spain,"POLYGON ((-2.18083 34.85145, -2.71695 34.77839...",0.0,0.0,0.025556,0.002060,0.0,0.000000,0.000000,0.000394,0.0,0.000108,0.0,0.028117
1,280,Malta,"POLYGON ((14.68659 35.53849, 14.14122 35.56449...",0.0,0.0,0.000000,0.000023,0.0,0.000008,0.000000,0.000054,0.0,0.000000,0.0,0.000085
2,363,Spain,"POLYGON ((-2.79499 35.22702, -3.33331 35.15029...",0.0,0.0,0.004915,0.000586,0.0,0.000000,0.000000,0.000031,0.0,0.000031,0.0,0.005563
3,412,Greece,"POLYGON ((23.95653 35.05738, 23.41935 35.13773...",0.0,0.0,0.043410,0.007299,0.0,0.000000,0.005347,0.000648,0.0,0.000000,0.0,0.056705
4,413,Greece,"POLYGON ((24.49271 34.97391, 23.95653 35.05738...",0.0,0.0,0.035139,0.004282,0.0,0.000123,0.016381,0.001258,0.0,0.000231,0.0,0.057415


In [24]:
# Guardar el resultado en un nuevo shapefile
#Grid_50.to_file('Data/GLOBIO/RCP2_6/Grid_H_4.shp')

In [25]:
# Guardar el resultado en un nuevo shapefile
Grid_50_s = Grid_50[['ET_ID', 'Area_H_4']]

#  Unir los DataFrames usando el campo ET_ID como clave
Grid_Habitats = pd.merge(Grid_Habitats, Grid_50_s, on='ET_ID', how='left')
Grid_Habitats

,ET_ID,COUNTRY,geometry,Area_H_1,Area_H_2,Area_H_3,Area_H_4
0,249,Spain,"POLYGON ((-2.18083 34.85145, -2.71695 34.77839...",0.087106,0.000046,0.000177,0.028117
1,280,Malta,"POLYGON ((14.68659 35.53849, 14.14122 35.56449...",0.224738,0.000000,0.000000,0.000085
2,363,Spain,"POLYGON ((-2.79499 35.22702, -3.33331 35.15029...",0.207083,0.000116,0.000031,0.005563
3,412,Greece,"POLYGON ((23.95653 35.05738, 23.41935 35.13773...",0.113827,0.004414,0.000139,0.056705
4,413,Greece,"POLYGON ((24.49271 34.97391, 23.95653 35.05738...",0.123380,0.001273,0.000000,0.057415
...,...,...,...,...,...,...,...
3101,9369,Norway,"POLYGON ((26.92771 70.58708, 25.63446 70.69412...",0.445810,0.003835,0.004298,0.114128
3102,9370,Norway,"POLYGON ((28.20775 70.47181, 26.92771 70.58708...",0.191852,0.017670,0.016150,0.271250
3103,9371,Norway,"POLYGON ((29.47384 70.34844, 28.20775 70.47181...",0.136142,0.007948,0.010324,0.285062
3104,9372,Norway,"POLYGON ((30.72534 70.21713, 29.47384 70.34844...",0.218349,0.003410,0.003356,0.235787


In [26]:
# Guardar el resultado en un nuevo shapefile
#Grid_Habitats.to_file('Data/GLOBIO/RCP2_6/Grid_Habitats.shp')

In [27]:
# Cerrar el dataset raster
src.close()

# % DE ÁREA DISPONIBLE POR CUADRÍCULA

In [28]:
# Calculo del area para cada cuadrícula en km²
Grid_Habitats['Area_grid'] = Grid_Habitats.geometry.area #en m2
Grid_Habitats

C:\Users\molic\AppData\Local\Temp\ipykernel_19688\850118656.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  Grid_Habitats['Area_grid'] = Grid_Habitats.geometry.area #en m2


,ET_ID,COUNTRY,geometry,Area_H_1,Area_H_2,Area_H_3,Area_H_4,Area_grid
0,249,Spain,"POLYGON ((-2.18083 34.85145, -2.71695 34.77839...",0.087106,0.000046,0.000177,0.028117,0.246970
1,280,Malta,"POLYGON ((14.68659 35.53849, 14.14122 35.56449...",0.224738,0.000000,0.000000,0.000085,0.249205
2,363,Spain,"POLYGON ((-2.79499 35.22702, -3.33331 35.15029...",0.207083,0.000116,0.000031,0.005563,0.248088
3,412,Greece,"POLYGON ((23.95653 35.05738, 23.41935 35.13773...",0.113827,0.004414,0.000139,0.056705,0.247813
4,413,Greece,"POLYGON ((24.49271 34.97391, 23.95653 35.05738...",0.123380,0.001273,0.000000,0.057415,0.247566
...,...,...,...,...,...,...,...,...
3101,9369,Norway,"POLYGON ((26.92771 70.58708, 25.63446 70.69412...",0.445810,0.003835,0.004298,0.114128,0.612105
3102,9370,Norway,"POLYGON ((28.20775 70.47181, 26.92771 70.58708...",0.191852,0.017670,0.016150,0.271250,0.608670
3103,9371,Norway,"POLYGON ((29.47384 70.34844, 28.20775 70.47181...",0.136142,0.007948,0.010324,0.285062,0.605030
3104,9372,Norway,"POLYGON ((30.72534 70.21713, 29.47384 70.34844...",0.218349,0.003410,0.003356,0.235787,0.601196


In [29]:
# Calculo porcentaje de habitat
Grid_Habitats['%_H_1'] = (Grid_Habitats['Area_H_1']*100)/Grid_Habitats['Area_grid']
Grid_Habitats['%_H_2'] = (Grid_Habitats['Area_H_2']*100)/Grid_Habitats['Area_grid']
Grid_Habitats['%_H_3'] = (Grid_Habitats['Area_H_3']*100)/Grid_Habitats['Area_grid']
Grid_Habitats['%_H_4'] = (Grid_Habitats['Area_H_4']*100)/Grid_Habitats['Area_grid']

Grid_Habitats

,ET_ID,COUNTRY,geometry,Area_H_1,Area_H_2,Area_H_3,Area_H_4,Area_grid,%_H_1,%_H_2,%_H_3,%_H_4
0,249,Spain,"POLYGON ((-2.18083 34.85145, -2.71695 34.77839...",0.087106,0.000046,0.000177,0.028117,0.246970,35.270045,0.018746,0.071859,11.384892
1,280,Malta,"POLYGON ((14.68659 35.53849, 14.14122 35.56449...",0.224738,0.000000,0.000000,0.000085,0.249205,90.181696,0.000000,0.000000,0.034059
2,363,Spain,"POLYGON ((-2.79499 35.22702, -3.33331 35.15029...",0.207083,0.000116,0.000031,0.005563,0.248088,83.471585,0.046653,0.012441,2.242455
3,412,Greece,"POLYGON ((23.95653 35.05738, 23.41935 35.13773...",0.113827,0.004414,0.000139,0.056705,0.247813,45.932766,1.781016,0.056046,22.882314
4,413,Greece,"POLYGON ((24.49271 34.97391, 23.95653 35.05738...",0.123380,0.001273,0.000000,0.057415,0.247566,49.837149,0.514267,0.000000,23.191884
...,...,...,...,...,...,...,...,...,...,...,...,...
3101,9369,Norway,"POLYGON ((26.92771 70.58708, 25.63446 70.69412...",0.445810,0.003835,0.004298,0.114128,0.612105,72.832347,0.626507,0.702141,18.645192
3102,9370,Norway,"POLYGON ((28.20775 70.47181, 26.92771 70.58708...",0.191852,0.017670,0.016150,0.271250,0.608670,31.519831,2.903009,2.653274,44.564356
3103,9371,Norway,"POLYGON ((29.47384 70.34844, 28.20775 70.47181...",0.136142,0.007948,0.010324,0.285062,0.605030,22.501694,1.313577,1.706374,47.115313
3104,9372,Norway,"POLYGON ((30.72534 70.21713, 29.47384 70.34844...",0.218349,0.003410,0.003356,0.235787,0.601196,36.319039,0.567284,0.558300,39.219634


In [30]:
Grid_Habitats_Variables = Grid_Habitats[['ET_ID', 'geometry', '%_H_1', '%_H_2', '%_H_3', '%_H_4']]
Grid_Habitats_Variables

,ET_ID,geometry,%_H_1,%_H_2,%_H_3,%_H_4
0,249,"POLYGON ((-2.18083 34.85145, -2.71695 34.77839...",35.270045,0.018746,0.071859,11.384892
1,280,"POLYGON ((14.68659 35.53849, 14.14122 35.56449...",90.181696,0.000000,0.000000,0.034059
2,363,"POLYGON ((-2.79499 35.22702, -3.33331 35.15029...",83.471585,0.046653,0.012441,2.242455
3,412,"POLYGON ((23.95653 35.05738, 23.41935 35.13773...",45.932766,1.781016,0.056046,22.882314
4,413,"POLYGON ((24.49271 34.97391, 23.95653 35.05738...",49.837149,0.514267,0.000000,23.191884
...,...,...,...,...,...,...
3101,9369,"POLYGON ((26.92771 70.58708, 25.63446 70.69412...",72.832347,0.626507,0.702141,18.645192
3102,9370,"POLYGON ((28.20775 70.47181, 26.92771 70.58708...",31.519831,2.903009,2.653274,44.564356
3103,9371,"POLYGON ((29.47384 70.34844, 28.20775 70.47181...",22.501694,1.313577,1.706374,47.115313
3104,9372,"POLYGON ((30.72534 70.21713, 29.47384 70.34844...",36.319039,0.567284,0.558300,39.219634


In [31]:
# Guardar el resultado en un nuevo shapefile
#Grid_Habitats_Variables.to_file('Data/GLOBIO/RCP2_6/Grid_Habitats_Variables.shp')

# UNION CON VARIABLES CLIMÁTICAS

Se tienen en cuenta solo las variables seleccionadas con el Jupyter Notebook 06_Analisis_variables_independientes.ipynb

In [2]:
#Cargar el grid
#Grid_Habitats_Variables = gpd.read_file(r'Data/GLOBIO/RCP2_6/Grid_Habitats_Variables.shp')
Grid_Habitats_Variables

,ET_ID,%_H_1,%_H_2,%_H_3,%_H_4,geometry
0,249,35.270045,0.018746,0.071859,11.384892,"POLYGON ((-2.18083 34.85145, -2.71695 34.77839..."
1,280,90.181696,0.000000,0.000000,0.034059,"POLYGON ((14.68659 35.53849, 14.14122 35.56449..."
2,363,83.471585,0.046653,0.012441,2.242455,"POLYGON ((-2.79499 35.22702, -3.33331 35.15029..."
3,412,45.932766,1.781016,0.056046,22.882314,"POLYGON ((23.95653 35.05738, 23.41935 35.13773..."
4,413,49.837149,0.514267,0.000000,23.191884,"POLYGON ((24.49271 34.97391, 23.95653 35.05738..."
...,...,...,...,...,...,...
3101,9369,72.832347,0.626507,0.702141,18.645192,"POLYGON ((26.92771 70.58708, 25.63446 70.69412..."
3102,9370,31.519831,2.903009,2.653274,44.564356,"POLYGON ((28.20775 70.47181, 26.92771 70.58708..."
3103,9371,22.501694,1.313577,1.706374,47.115313,"POLYGON ((29.47384 70.34844, 28.20775 70.47181..."
3104,9372,36.319039,0.567284,0.558300,39.219634,"POLYGON ((30.72534 70.21713, 29.47384 70.34844..."


In [4]:
import glob
import os

directorio = 'Data/CHELSA/ESCENARIOS_FUTUROS/Semejante_RCP_2_6/Estadistica_Zona' 
archivos_shp = glob.glob(directorio + '/Bio_*.shp') 
archivos_shp

['C:/Users/molic/Desktop/TFM_V1/Chelsa_bio/ESCENARIOS_FUTUROS/Semejante_RCP_2_6/Estadistica_Zona\\Bio_01.shp',
 'C:/Users/molic/Desktop/TFM_V1/Chelsa_bio/ESCENARIOS_FUTUROS/Semejante_RCP_2_6/Estadistica_Zona\\Bio_02.shp',
 'C:/Users/molic/Desktop/TFM_V1/Chelsa_bio/ESCENARIOS_FUTUROS/Semejante_RCP_2_6/Estadistica_Zona\\Bio_03.shp',
 'C:/Users/molic/Desktop/TFM_V1/Chelsa_bio/ESCENARIOS_FUTUROS/Semejante_RCP_2_6/Estadistica_Zona\\Bio_04.shp',
 'C:/Users/molic/Desktop/TFM_V1/Chelsa_bio/ESCENARIOS_FUTUROS/Semejante_RCP_2_6/Estadistica_Zona\\Bio_05.shp',
 'C:/Users/molic/Desktop/TFM_V1/Chelsa_bio/ESCENARIOS_FUTUROS/Semejante_RCP_2_6/Estadistica_Zona\\Bio_08.shp',
 'C:/Users/molic/Desktop/TFM_V1/Chelsa_bio/ESCENARIOS_FUTUROS/Semejante_RCP_2_6/Estadistica_Zona\\Bio_12.shp',
 'C:/Users/molic/Desktop/TFM_V1/Chelsa_bio/ESCENARIOS_FUTUROS/Semejante_RCP_2_6/Estadistica_Zona\\Bio_15.shp']

In [5]:
# Cargar cada archivo shapefile y asignarlo a una variable con el nombre del archivo
for shp in archivos_shp:
    # Obtener el nombre base del archivo sin la extensión
    file_name = os.path.splitext(os.path.basename(shp))[0]
    
    # Cargar el shapefile en un GeoDataFrame
    gdf = gpd.read_file(shp)
    
    # Asignar el GeoDataFrame a una variable con el nombre del archivo
    globals()[file_name] = gdf

In [6]:
print(Bio_12)

      ET_ID COUNTRY   x   y   Bio_12_mea  \
0       249   Spain  -2  35   312.114703   
1       280   Malta  14  36   487.709028   
2       363   Spain  -3  35   342.546921   
3       412  Greece  24  35   857.540751   
4       413  Greece  24  35   780.459725   
...     ...     ...  ..  ..          ...   
3101   9369  Norway  26  71   896.691386   
3102   9370  Norway  28  71  1014.888505   
3103   9371  Norway  29  71   960.570335   
3104   9372  Norway  30  70   862.157861   
3105   9485  Norway  28  71   807.645176   

                                               geometry  
0     POLYGON ((-2.18083 34.85145, -2.71695 34.77839...  
1     POLYGON ((14.68659 35.53849, 14.14122 35.56449...  
2     POLYGON ((-2.79499 35.22702, -3.33331 35.15029...  
3     POLYGON ((23.95653 35.05738, 23.41935 35.13773...  
4     POLYGON ((24.49271 34.97391, 23.95653 35.05738...  
...                                                 ...  
3101  POLYGON ((26.92771 70.58708, 25.63446 70.69412...  
3102  P

In [7]:
# Guardar el resultado en un nuevo shapefile
Bio_01 = Bio_01[['ET_ID', 'Bio_01_mea']]
Bio_02 = Bio_02[['ET_ID', 'Bio_02_mea']]
Bio_03 = Bio_03[['ET_ID', 'Bio_03_mea']]
Bio_04 = Bio_04[['ET_ID', 'Bio_04_mea']]
Bio_05 = Bio_05[['ET_ID', 'Bio_05_mea']]
Bio_08 = Bio_08[['ET_ID', 'Bio_08_mea']]
Bio_12 = Bio_12[['ET_ID', 'Bio_12_mea']]
Bio_15 = Bio_15[['ET_ID', 'Bio_15_mea']]


In [8]:
df_names = ['Bio_01', 'Bio_02', 'Bio_03', 'Bio_04', 'Bio_05', 'Bio_08', 'Bio_12', 'Bio_15']

In [9]:
for df_name in df_names:
    df = globals()[df_name] 
    Grid_Habitats_Variables = pd.merge(Grid_Habitats_Variables, df, on='ET_ID', how='left')

Grid_Habitats_Variables.head(2)

,ET_ID,%_H_1,%_H_2,%_H_3,%_H_4,geometry,Bio_01_mea,Bio_02_mea,Bio_03_mea,Bio_04_mea,Bio_05_mea,Bio_08_mea,Bio_12_mea,Bio_15_mea
0,249,35.270045,0.018746,0.071859,11.384892,"POLYGON ((-2.18083 34.85145, -2.71695 34.77839...",19.603753,8.926230,0.367492,469.222097,31.043674,15.153036,312.114703,56.551026
1,280,90.181696,0.000000,0.000000,0.034059,"POLYGON ((14.68659 35.53849, 14.14122 35.56449...",20.259975,1.772527,0.117356,452.822427,28.277055,16.988451,487.709028,86.413598
2,363,83.471585,0.046653,0.012441,2.242455,"POLYGON ((-2.79499 35.22702, -3.33331 35.15029...",19.564446,3.937430,0.241514,372.215790,27.388354,15.334658,342.546921,62.331019
3,412,45.932766,1.781016,0.056046,22.882314,"POLYGON ((23.95653 35.05738, 23.41935 35.13773...",19.066592,4.526317,0.221784,530.199328,29.293554,13.749383,857.540751,95.105437
4,413,49.837149,0.514267,0.000000,23.191884,"POLYGON ((24.49271 34.97391, 23.95653 35.05738...",18.994794,4.130676,0.206098,536.570109,28.752217,13.267485,780.459725,92.462110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3101,9369,72.832347,0.626507,0.702141,18.645192,"POLYGON ((26.92771 70.58708, 25.63446 70.69412...",3.177863,3.998173,0.204703,550.431335,13.989053,3.769566,896.691386,24.014017
3102,9370,31.519831,2.903009,2.653274,44.564356,"POLYGON ((28.20775 70.47181, 26.92771 70.58708...",1.448917,4.985950,0.205812,682.670361,14.551232,7.223269,1014.888505,23.333436
3103,9371,22.501694,1.313577,1.706374,47.115313,"POLYGON ((29.47384 70.34844, 28.20775 70.47181...",1.047464,4.884496,0.201313,686.225362,14.151825,8.186344,960.570335,24.344985
3104,9372,36.319039,0.567284,0.558300,39.219634,"POLYGON ((30.72534 70.21713, 29.47384 70.34844...",2.073357,3.985157,0.193693,579.416969,13.068575,8.788651,862.157861,23.256093


In [10]:
# Redondear todas las columnas float64 a 4 decimales
Grid_Habitats_Variables = Grid_Habitats_Variables.round(4)

In [11]:
# Asegurarse de que no hay duplicados
Grid_Habitats_Variables = Grid_Habitats_Variables.drop_duplicates(subset=['ET_ID'])
Grid_Habitats_Variables.head(2)

,ET_ID,%_H_1,%_H_2,%_H_3,%_H_4,geometry,Bio_01_mea,Bio_02_mea,Bio_03_mea,Bio_04_mea,Bio_05_mea,Bio_08_mea,Bio_12_mea,Bio_15_mea
0,249,35.2700,0.0187,0.0719,11.3849,"POLYGON ((-2.18083 34.85145, -2.71695 34.77839...",19.6038,8.9262,0.3675,469.2221,31.0437,15.1530,312.1147,56.5510
1,280,90.1817,0.0000,0.0000,0.0341,"POLYGON ((14.68659 35.53849, 14.14122 35.56449...",20.2600,1.7725,0.1174,452.8224,28.2771,16.9885,487.7090,86.4136
2,363,83.4716,0.0467,0.0124,2.2425,"POLYGON ((-2.79499 35.22702, -3.33331 35.15029...",19.5644,3.9374,0.2415,372.2158,27.3884,15.3347,342.5469,62.3310
3,412,45.9328,1.7810,0.0560,22.8823,"POLYGON ((23.95653 35.05738, 23.41935 35.13773...",19.0666,4.5263,0.2218,530.1993,29.2936,13.7494,857.5408,95.1054
4,413,49.8371,0.5143,0.0000,23.1919,"POLYGON ((24.49271 34.97391, 23.95653 35.05738...",18.9948,4.1307,0.2061,536.5701,28.7522,13.2675,780.4597,92.4621
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3101,9369,72.8323,0.6265,0.7021,18.6452,"POLYGON ((26.92771 70.58708, 25.63446 70.69412...",3.1779,3.9982,0.2047,550.4313,13.9891,3.7696,896.6914,24.0140
3102,9370,31.5198,2.9030,2.6533,44.5644,"POLYGON ((28.20775 70.47181, 26.92771 70.58708...",1.4489,4.9860,0.2058,682.6704,14.5512,7.2233,1014.8885,23.3334
3103,9371,22.5017,1.3136,1.7064,47.1153,"POLYGON ((29.47384 70.34844, 28.20775 70.47181...",1.0475,4.8845,0.2013,686.2254,14.1518,8.1863,960.5703,24.3450
3104,9372,36.3190,0.5673,0.5583,39.2196,"POLYGON ((30.72534 70.21713, 29.47384 70.34844...",2.0734,3.9852,0.1937,579.4170,13.0686,8.7887,862.1579,23.2561


In [12]:
# Redondear todas las columnas float64 a 4 decimales
Grid_Habitats_Variables = Grid_Habitats_Variables.round(4)
Grid_Habitats_Variables.head(3)

,ET_ID,%_H_1,%_H_2,%_H_3,%_H_4,geometry,Bio_01_mea,Bio_02_mea,Bio_03_mea,Bio_04_mea,Bio_05_mea,Bio_08_mea,Bio_12_mea,Bio_15_mea
0,249,35.2700,0.0187,0.0719,11.3849,"POLYGON ((-2.18083 34.85145, -2.71695 34.77839...",19.6038,8.9262,0.3675,469.2221,31.0437,15.1530,312.1147,56.5510
1,280,90.1817,0.0000,0.0000,0.0341,"POLYGON ((14.68659 35.53849, 14.14122 35.56449...",20.2600,1.7725,0.1174,452.8224,28.2771,16.9885,487.7090,86.4136
2,363,83.4716,0.0467,0.0124,2.2425,"POLYGON ((-2.79499 35.22702, -3.33331 35.15029...",19.5644,3.9374,0.2415,372.2158,27.3884,15.3347,342.5469,62.3310


In [14]:
# Exportar el resultado 
Grid_Habitats_Variables.to_csv('Data/Variables/RCP2_6/Grid_Variables_RCP2_6.csv', index=False, decimal='.')
Grid_Habitats_Variables.to_file('Data/Variables/RCP2_6/Grid_Variables_RCP2_6.shp')
